# 🎬 ViarteIA - Ultra-Automated Full Stack (v2.3)

**Un solo clic para dominar la IA de Video.**

---

In [ ]:
#@title 🚀 1. Configuración Total (Drive + Deps + Repo)
#@markdown Solo ejecuta esta celda y espera a que termine (3-7 min).

from google.colab import drive
import os
import sys

print("📂 1/4 Montando Google Drive...")
drive.mount('/content/drive')

PERSISTENT_DIR = "/content/drive/MyDrive/ViarteIA_Data"
os.makedirs(f"{PERSISTENT_DIR}/models", exist_ok=True)
os.makedirs(f"{PERSISTENT_DIR}/db", exist_ok=True)

print("🛠️ 2/4 Actualizando sistema (Ffmpeg, Node.js)...")
!apt-get update -qq && apt-get install -y ffmpeg aria2 > /dev/null
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash - > /dev/null
!sudo apt-get install -y nodejs > /dev/null
!npm install -g pnpm > /dev/null

print("🐍 3/4 Instalando librerías de IA (esto tarda 2-4 min)...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors xformers insightface onnxruntime-gpu realesrgan basicsr facexlib fastapi uvicorn groq python-dotenv pyngrok python-multipart

# Patch para BasicSR
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py

print("⬇️ 4/4 Sincronizando repositorio y Node modules...")
%cd /content
if not os.path.exists('ViarteIA'):
    !git clone -q https://github.com/Juanpalojime/ViarteIA.git
else:
    %cd ViarteIA
    !git pull -q
    %cd ..

%cd ViarteIA
!mkdir -p backend/python-ai/src/models
!rm -rf backend/python-ai/src/models
!ln -s {PERSISTENT_DIR}/models backend/python-ai/src/models

%cd backend/node-api
!pnpm install --legacy-peer-deps -s > /dev/null

print("\n✅ ¡ENTORNO LISTO! Continúa con la siguiente celda.")

In [ ]:
#@title 📥 2. Descargar Modelos AI (Persistent)
MODEL_PATH = f"{PERSISTENT_DIR}/models"

def download_if_missing(name, path, cmd):
    if not os.path.exists(path):
        print(f"📥 Descargando {name}...")
        os.system(cmd)
    else:
        print(f"✔️ {name} ya está en Drive.")

download_if_missing("SVD-XT", f"{MODEL_PATH}/svd-xt", f"huggingface-cli download stabilityai/stable-video-diffusion-img2vid-xt --local-dir {MODEL_PATH}/svd-xt --local-dir-use-symlinks False")
download_if_missing("Real-ESRGAN", f"{MODEL_PATH}/RealESRGAN_x2plus.pth", f"wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P {MODEL_PATH}")

print("✅ Verificación de modelos completada.")

In [ ]:
#@title 🔑 3. Configurar APIs y Secretos
GROQ_API_KEY = "" #@param {type:"string"}
NGROK_AUTHTOKEN = "" #@param {type:"string"}
JWT_SECRET = "viarteia_master_key_998877" #@param {type:"string"}

if not GROQ_API_KEY or not NGROK_AUTHTOKEN:
    raise ValueError("❌ Debes ingresar las claves de Groq y Ngrok")

import os

env_content = f"""
PORT=3001
GROQ_API_KEY={GROQ_API_KEY}
JWT_SECRET={JWT_SECRET}
DATABASE_URL={PERSISTENT_DIR}/db/dev.db
PYTHON_API_URL=http://localhost:8000
"""
with open('/content/ViarteIA/backend/node-api/.env', 'w') as f:
    f.write(env_content)

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["NGROK_TOKEN"] = NGROK_AUTHTOKEN
print("✅ Claves configuradas correctamente.")

In [ ]:
#@title 🌐 4. Iniciar Backend y Sincronizar Frontend
import subprocess
import time
import os
from pyngrok import ngrok

print("🚀 Iniciando túnel Ngrok (vía 127.0.0.1)...")
ngrok.kill()
ngrok.set_auth_token(os.environ["NGROK_TOKEN"])
public_url = ngrok.connect("127.0.0.1:3001").public_url
print(f"🌍 BACKEND URL: {public_url}")

print("🔗 Inyectando URL en Frontend...")
with open('/content/ViarteIA/apps/web/.env.local', 'w') as f:
    f.write(f"VITE_API_URL={public_url}/api\n")

print("🧠 Arrancando Motor Python...")
subprocess.Popen(["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"], 
                 cwd="/content/ViarteIA/backend/python-ai/src",
                 stdout=open("/content/python_api.log", "w"), 
                 stderr=subprocess.STDOUT)

time.sleep(10)

print("⚡ Arrancando API Node.js...")
subprocess.Popen(["pnpm", "run", "dev"], 
                 cwd="/content/ViarteIA/backend/node-api",
                 stdout=open("/content/node_api.log", "w"), 
                 stderr=subprocess.STDOUT)

time.sleep(10)
print("\n✨ ¡LISTO! Todo está conectado.")
print(f"Usa esta URL en tu navegador o cliente: {public_url}")

In [ ]:
#@title 📋 Ver Logs del Backend
#@markdown Ejecuta esta celda si tienes problemas de conexión.
print("--- ULTIMOS LOGS DE NODE.JS ---")
!tail -n 20 /content/node_api.log
print("\n--- ULTIMOS LOGS DE PYTHON ---")
!tail -n 20 /content/python_api.log